# Advanced Querying Mongo

Importing libraries and setting up connection

In [2]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")

db = client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [4]:
# Your Code

query = {'name':'Babelgum'}
data = list(db['companies'].find(query, {'name':1, '_id':0}))

print(data)

[{'name': 'Babelgum'}]


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [8]:
# Your Code

query = {'number_of_employees':{'$gt':5000}}
data = list(db['companies'].find(query).sort('number_of_employees').limit(20))
len(data)

20

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [9]:
# Your Code

query = {'$and':[{'founded_year':{'$gte':2000}}, {'founded_year':{'$lte':2005}}]}
data = list(db['companies'].find(query, {'name':1, 'founded_year':1, '_id':0}))
len(data)

3734

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [10]:
# Your Code

query = {'$and':[{"ipo.valuation_amount":{'$gt':100000000}}, {'founded_year':{'$lt':2010}}]}
data = list(db['companies'].find(query, {'name':1, 'ipo':1, '_id':0}))
len(data)

42

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [11]:
# Your Code

query = {'$and':[{'number_of_employees':{'$lt':1000}}, {'founded_year':{'$lt':2005}}]}
data = list(db['companies'].find(query, {'name':1, 'number_of_employees':1, '_id': 0}).sort('number_of_employees', -1).limit(10))
data


[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'Cornerstone OnDemand', 'number_of_employees': 881},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'ZoomInfo', 'number_of_employees': 800},
 {'name': 'MySpace', 'number_of_employees': 800}]

### 6. All the companies that don't include the `partners` field.

In [12]:
# Your Code

query = {'partners':{'$exists':True, '$not':{'$size':0}}}
data = list(db['companies'].find(query))
len(data)

154

### 7. All the companies that have a null type of value on the `category_code` field.

In [13]:
# Your Code

query = {'category_code':{'$type':'null'}}
data = list(db['companies'].find(query))
len(data)

2751

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [14]:
# Your Code

query = {'$and':[{'number_of_employees':{'$gte':100}},{'number_of_employees':{'$lt':1000}}]}
data = list(db['companies'].find(query, {'name':1, 'number_of_employees':1, '_id':0}))
len(data)

917

### 9. Order all the companies by their IPO price in a descending order.

In [27]:
# Your Code

query = {'ipo.valuation_amount':{'$exists':True}}
data = list(db['companies'].find(query).sort([('ipo.value_amount',-1)]))
len(data)

390

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [29]:
# Your Code

query = {'number_of_employees': {'$exists':True}}
data10 = db["companies"].find(query).sort("number_of_employees",-1).limit(10)

len(list(data10))

10

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [32]:
# Your Code

query = {'founded_month':{'$gte':7, '$lt':12}}
data = list(db['companies'].find(query).sort('numbre_of_employees',1).limit(1000))
len(data)

1000

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [34]:
# Your Code

query = {'$and':[{'founded_year':{'$lt':2000}}, {'acquisition.price_amount':{'$gt':1000000}}]}
data = list(db['companies'].find(query))
len(data)

224

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [38]:
# Your Code

query = {'acquisition.acquired_year':{'$gt':2010}}
data = list(db['companies'].find(query, {'name':1, 'acquisition':1, '_id':0}).sort([('acquisition.price_amount',1)]))
len(data)

736

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [45]:
# Your Code

query = {'founded_year':{'$exists':True, '$not':{'$size':0}}}
data = list(db['companies'].find(query, {'name':1, 'founded_year':1, '_id':0}).sort([('founded_year',1)]).limit(1000))
len(data)

1000

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code



### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code